# De la bonne découpe pour un code source

In [2]:
document = open("/home/thomas/raglab/misc/thomas/example.py", "r")
text = document.read()
document.close()

## Approche sémantique

### Outil pour tokeniser du code

In [3]:
import code_tokenize as ctok # pip install code-tokenize

tokens = ctok.tokenize(text, lang = "python")
print(tokens[:10])

[class, AnimalBase, :, #INDENT#, def, __init__, (, self, ,, name]


/home/thomas/.local/lib/python3.9/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


### Tree-sitter permet de déduire un graphe syntaxique d'un code source

On construit un arbre syntaxique avec le code du app.py

In [4]:
import tree_sitter_python as tspython
from tree_sitter import Language, Parser

PY_LANGUAGE = Language(tspython.language(), "python")
parser = Parser()
parser.set_language(PY_LANGUAGE)
tree = parser.parse(bytes(text, "utf8"))

In [19]:
dir(tree)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'changed_ranges',
 'edit',
 'included_ranges',
 'root_node',
 'root_node_with_offset',
 'text',
 'walk']

In [6]:
def node_content(node) :
    return text[node.start_byte:node.end_byte]

In [7]:
print(node_content(tree.root_node))
print("\n\n\nNumber or children\n-----------")
print(len(tree.root_node.children))

class AnimalBase:
    def __init__(self, name, energy=100, position=(0, 0), health=100):
        self.name = name
        self.energy = energy
        self.position = position
        self.health = health

    def move(self, dx, dy):
        if self.energy > 0:
            self.position = (self.position[0] + dx, self.position[1] + dy)
            self.energy -= 1
        else:
            print(f"{self.name} is too tired to move.")

    def eat(self, food):
        if isinstance(food, AnimalBase):
            print(f"{self.name} cannot eat other animals!")
        else:
            self.energy += 10

    def sleep(self, duration):
        if self.energy < 100:
            self.energy += duration
            self.energy = min(self.energy, 100)
            print(f"{self.name} slept and now has {self.energy} energy.")

    def attack(self, animal):
        if self.energy > 0 and self.health > 0:
            damage = 10
            animal.health -= damage
            self.energy -= 5
     

In [8]:
print(node_content(tree.root_node.children[1]))

class Vertebre(AnimalBase):
    def __init__(self, name, bone_density, *args, **kwargs):
        super().__init__(name, *args, **kwargs)
        self.bone_density = bone_density

    def attack(self, animal):
        super().attack(animal)
        if self.bone_density > 50:
            extra_damage = 5
            animal.health -= extra_damage
            print(f"{self.name} uses strong bones to cause extra {extra_damage} damage.")


## Approche naïve

In [9]:
def cut_document(document) -> list[str] :
    segment_size = 300
    segments = []
    for i in range(0, len(document), segment_size) :
        segments.append(document[i:i+segment_size])
    return segments

document = "A"*(300*3+299)

assert len(cut_document(document)) == 4

In [10]:

segments = cut_document(text)

In [17]:
import chromadb
from random import randint

client = chromadb.HttpClient(
    host="149.202.47.109",
    port="45000",
    settings=chromadb.Settings(
        chroma_client_auth_provider="chromadb.auth.token.TokenAuthClientProvider",
        chroma_client_auth_credentials="",
    ),
)

try :
    client.delete_collection("segmentation_test")
except : 
    pass

collection = client.get_or_create_collection("segmentation_test", embedding_function=chromadb.utils.embedding_functions.Text2VecEmbeddingFunction())
collection.add(documents=segments, ids=[str(randint(0, 10000)) for x in range(len(segments))])

2024-05-03 07:44:53.219 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cpu


In [18]:
print(segments[5])

e_density > 50:
            extra_damage = 5
            animal.health -= extra_damage
            print(f"{self.name} uses strong bones to cause extra {extra_damage} damage.")

class Mammifere(Vertebre):
    def __init__(self, name, fur_type, *args, **kwargs):
        super().__init__(name, *args, 


In [19]:
result = collection.query(query_texts=["Implémente un animal qui serait le résultat d'une mutation entre deux Luminherbe"], n_results=3)

In [20]:
result.keys()

dict_keys(['ids', 'distances', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])

In [21]:
for d in result["documents"][0] :
    print("\n\n")
    print(d)




attack(self, animal):
        super().attack(animal)
        if 'fire' in self.get_environment():  # Assuming get_environment method returns elements in environment
            animal.health -= 20  # Extra damage in a fiery environment

    def eat(self, animal):
        super().eat(animal)
        



l)

class Arborevore(AnimalBase):
    def eat(self, food):
        if food == 'tree':
            self.energy += 15
            print(f"{self.name} eats a tree and gains energy.")
        else:
            print(f"{self.name} can only eat trees.")

# Final implementation classes
class Luminherbe(Mam



if animal.health <= 0:
            self.energy += 5  # Extra energy gain for successful hunt

# Utility methods
def get_time():
    return 'day'  # Stub to simulate day or night

def get_environment():
    return ['forest', 'fire']  # Stub to simulate environmental conditions

# CTF-like hidden flag


L'exemple précédent montre un exemple d'erreur à cause d'une mauvaise découpe : on a pas l'implémentation du Luminherbe. Aussi on aurait voulu avoir des informations sur la classe de base Mammifère.

## Mix des deux

L'approche sémantique est intuitivement plus qualitative car elle permet d'aller chercher les informations liées sémantiquement et non positionnelement.

En revanche, l'approche sémanque est difficile à employer seule, en partant d'une requête : là où un bloc d'information aura plus de chance de matcher avec notre requête, un bloc d'information de la taille d'un token le sera moins

In [24]:
client.delete_collection("segmentation_test_tree")

In [25]:
def store_tree_into_chroma(collection, tree) :
    if tree.child_count == 0 :
        return
    for child in tree.children :
        store_tree_into_chroma(collection, child)
    collection.add(documents=[node_content(tree)], ids=[str(randint(0, 10000))])

collection = client.get_or_create_collection("segmentation_test_tree", embedding_function=chromadb.utils.embedding_functions.Text2VecEmbeddingFunction())
store_tree_into_chroma(collection, tree.root_node)

2024-05-03 07:49:24.157 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cpu


In [26]:
collection_tree = collection
collection = client.get_or_create_collection("segmentation_test", embedding_function=chromadb.utils.embedding_functions.Text2VecEmbeddingFunction())


2024-05-03 07:50:56.051 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cpu


In [30]:
print("Tree document :", collection_tree.count())
print("Chunk document: ", collection.count())

Tree document : 398
Chunk document:  14


In [32]:
result = collection_tree.query(query_texts=["Implémente un animal qui serait le résultat d'une mutation entre deux Luminherbe"], n_results=3)
for d in result["documents"][0] :
    print("\n\n")
    print(d)




class Carnivore(Mammifere):
    def eat(self, animal):
        if isinstance(animal, AnimalBase):
            if self.energy < 50:
                self.energy += 20
                animal.health = 0
                print(f"{self.name} eats {animal.name} and gains energy.")
            else:
                print(f"{self.name} is not hungry.")
        else:
            super().eat(animal)



def attack(self, animal):
        super().attack(animal)
        if 'fire' in self.get_environment():  # Assuming get_environment method returns elements in environment
            animal.health -= 20  # Extra damage in a fiery environment



class VoraceFeu(Carnivore):
    def __init__(self):
        super().__init__('VoraceFeu', 'rough', bone_density=80)

    def attack(self, animal):
        super().attack(animal)
        if 'fire' in self.get_environment():  # Assuming get_environment method returns elements in environment
            animal.health -= 20  # Extra damage in a fiery environment

[]